<a href="https://colab.research.google.com/github/CRosero/aml-project/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/gdrive/')

Mounted at /gdrive/


In [2]:
# Clone the entire repo.
!git clone -l -s git://github.com/CRosero/aml-project.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 85 (delta 21), reused 0 (delta 0), pack-reused 49
Receiving objects: 100% (85/85), 53.81 KiB | 1.05 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/cloned-repo
dataset     eval.py  model	train.ipynb  utils.py
eval.ipynb  loss.py  README.md	train.py


In [3]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 5.3 MB/s 


In [4]:
import argparse
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from model.build_BiSeNet import BiSeNet
import torch
from tensorboardX import SummaryWriter
from tqdm import tqdm
import numpy as np
from utils import poly_lr_scheduler
from utils import reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu
from loss import DiceLoss
import torch.cuda.amp as amp
import os
import os.path as osp
import random
import matplotlib.pyplot as plt
import collections
import torchvision
from torchvision.transforms import InterpolationMode
from torch.utils import data
from PIL import Image
import json

In [5]:
def label_mapping(input, mapping):
    output = np.copy(input)
    for ind in range(len(mapping)):
        output[input == mapping[ind][0]] = mapping[ind][1]
    return np.array(output, dtype=np.int64)

In [6]:
class cityscapesDataSet(data.Dataset):
    def __init__(self, root, list_path, max_iters=None, crop_size=(321, 321), mean=(128, 128, 128), scale=True, mirror=True, ignore_label=255):
        self.root = root
        self.list_path = list_path
        self.crop_size = crop_size
        self.scale = scale
        self.ignore_label = ignore_label
        self.mean = mean
        self.is_mirror = mirror
        # self.mean_bgr = np.array([104.00698793, 116.66876762, 122.67891434])
        self.img_ids = [i_id.strip() for i_id in open(list_path)]
        if not max_iters==None:
            self.img_ids = self.img_ids * int(np.ceil(float(max_iters) / len(self.img_ids)))
        self.files = []
        # for split in ["train", "trainval", "val"]:
        for name in self.img_ids:
            # Replace 'train' folder with 'images' folder and  
            # remove reference to city folder for images provided subset of the dataset
            img_file = osp.join(self.root[0].replace('train', 'images'), name.split("/", 1)[1])
            # Replace 'train' folder with 'labels' folder and  
            # remove reference to city folder for images provided subset of the dataset
            # rename file to be that of the label
            label_file = osp.join(self.root[0].replace('train', 'labels'), 
                                  name.split("/", 1)[1].replace('leftImg8bit', 'gtFine_labelIds'))
            self.files.append({
                "img": img_file,
                "label":label_file,
                "name": name
            })
        with open('/gdrive/MyDrive/data/Cityscapes/info.json', 'r') as fp:
            info = json.load(fp)
        self.mapping = np.array(info['label2train'], dtype=np.int)



    def __len__(self):
        return len(self.files)


    def __getitem__(self, index):
        datafiles = self.files[index]
        
        image = Image.open(datafiles["img"]).convert('RGB')
        label = Image.open(datafiles["label"])
        name = datafiles["name"]

        # resize
        print(f'image before resize: {image}')
        image = image.resize(self.crop_size, Image.BILINEAR)
        print(f'image after resize: {image}')
        print(f'label before resize: {label}')
        label = label.resize(self.crop_size, Image.NEAREST)
        print(f'label after resize: {label}')
        image = np.asarray(image, np.float32)
        print(f'image as array: {image}')
        label = np.asarray(label, np.float32)
        print(f'label as array: {label}')
        label = label_mapping(label, self.mapping)
        print(f'label after mapping: {label}')
        

        # re-assign labels to match the format of Cityscapes
        #label_copy = 255 * np.ones(label.shape, dtype=np.float32)
        #for k, v in self.id_to_trainid.items():
        #   label_copy[label == k] = v

        size = image.shape
        image = image[:, :, ::-1]  # change to BGR
        image -= self.mean
        image = image.transpose((2, 0, 1))

        return image.copy(), label.copy()


In [7]:
img_ids = [i_id.strip() for i_id in open('/gdrive/MyDrive/data/Cityscapes/train.txt')]
print(img_ids)

['strasbourg/strasbourg_000000_017593_leftImg8bit.png', 'zurich/zurich_000066_000019_leftImg8bit.png', 'hanover/hanover_000000_032681_leftImg8bit.png', 'ulm/ulm_000066_000019_leftImg8bit.png', 'dusseldorf/dusseldorf_000006_000019_leftImg8bit.png', 'hamburg/hamburg_000000_068916_leftImg8bit.png', 'hamburg/hamburg_000000_095561_leftImg8bit.png', 'monchengladbach/monchengladbach_000000_035364_leftImg8bit.png', 'weimar/weimar_000089_000019_leftImg8bit.png', 'aachen/aachen_000158_000019_leftImg8bit.png']


In [8]:
def val(args, model, dataloader):
    print('start val!')
    # label_info = get_label_info(csv_path)
    with torch.no_grad():
        model.eval()
        precision_record = []
        hist = np.zeros((args.num_classes, args.num_classes))
        for i, (data, label) in enumerate(dataloader):
            label = label.type(torch.LongTensor)
            data = data.cuda()
            label = label.long().cuda()

            # get RGB predict image
            predict = model(data).squeeze()
            predict = reverse_one_hot(predict)
            predict = np.array(predict.cpu())

            # get RGB label image
            label = label.squeeze()
            if args.loss == 'dice':
                label = reverse_one_hot(label)
            label = np.array(label.cpu())

            # compute per pixel accuracy
            precision = compute_global_accuracy(predict, label)
            hist += fast_hist(label.flatten(), predict.flatten(), args.num_classes)

            # there is no need to transform the one-hot array to visual RGB array
            # predict = colour_code_segmentation(np.array(predict), label_info)
            # label = colour_code_segmentation(np.array(label), label_info)
            precision_record.append(precision)
        
        precision = np.mean(precision_record)
        # miou = np.mean(per_class_iu(hist))
        miou_list = per_class_iu(hist)[:-1]
        # miou_dict, miou = cal_miou(miou_list, csv_path)
        miou = np.mean(miou_list)
        print('precision per pixel for test: %.3f' % precision)
        print('mIoU for validation: %.3f' % miou)
        # miou_str = ''
        # for key in miou_dict:
        #     miou_str += '{}:{},\n'.format(key, miou_dict[key])
        # print('mIoU for each class:')
        # print(miou_str)
        return precision, miou

In [9]:
def train(args, model, optimizer, dataloader_train, dataloader_val):
    writer = SummaryWriter(comment=''.format(args.optimizer, args.context_path))

    scaler = amp.GradScaler()

    if args.loss == 'dice':
        loss_func = DiceLoss()
    elif args.loss == 'crossentropy':
        loss_func = torch.nn.CrossEntropyLoss(ignore_index=255)
    max_miou = 0
    step = 0
    for epoch in range(args.num_epochs):
        lr = poly_lr_scheduler(optimizer, args.learning_rate, iter=epoch, max_iter=args.num_epochs)
        model.train()
        tq = tqdm(total=len(dataloader_train) * args.batch_size)
        tq.set_description('epoch %d, lr %f' % (epoch, lr))
        loss_record = []
        for i, (data, label) in enumerate(dataloader_train):
            print(f'data: {data}')
            print(f'label: {label}')
            data = data.cuda()
            label = label.long().cuda()
            optimizer.zero_grad()
            
            with amp.autocast():
                output, output_sup1, output_sup2 = model(data)
                loss1 = loss_func(output, label)
                loss2 = loss_func(output_sup1, label)
                loss3 = loss_func(output_sup2, label)
                loss = loss1 + loss2 + loss3
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            tq.update(args.batch_size)
            tq.set_postfix(loss='%.6f' % loss)
            step += 1
            writer.add_scalar('loss_step', loss, step)
            loss_record.append(loss.item())
        tq.close()
        loss_train_mean = np.mean(loss_record)
        writer.add_scalar('epoch/loss_epoch_train', float(loss_train_mean), epoch)
        print('loss for train : %f' % (loss_train_mean))
        if epoch % args.checkpoint_step == 0 and epoch != 0:
            import os
            if not os.path.isdir(args.save_model_path):
                os.mkdir(args.save_model_path)
            torch.save(model.module.state_dict(),
                       os.path.join(args.save_model_path, 'latest_dice_loss.pth'))

        if epoch % args.validation_step == 0 and epoch != 0:
            precision, miou = val(args, model, dataloader_val)
            if miou > max_miou:
                max_miou = miou
                import os 
                os.makedirs(args.save_model_path, exist_ok=True)
                torch.save(model.module.state_dict(),
                           os.path.join(args.save_model_path, 'best_dice_loss.pth'))
            writer.add_scalar('epoch/precision_val', precision, epoch)
            writer.add_scalar('epoch/miou val', miou, epoch)

In [10]:
def main(params):
    # basic parameters
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_epochs', type=int, default=300, help='Number of epochs to train for')
    parser.add_argument('--epoch_start_i', type=int, default=0, help='Start counting epochs from this number')
    parser.add_argument('--checkpoint_step', type=int, default=10, help='How often to save checkpoints (epochs)')
    parser.add_argument('--validation_step', type=int, default=10, help='How often to perform validation (epochs)')
    parser.add_argument('--dataset', type=str, default="Cityscapes", help='Dataset you are using.')
    parser.add_argument('--crop_height', type=int, default=720, help='Height of cropped/resized input image to network')
    parser.add_argument('--crop_width', type=int, default=960, help='Width of cropped/resized input image to network')
    parser.add_argument('--batch_size', type=int, default=32, help='Number of images in each batch')
    parser.add_argument("--iter-size", type=int, default=1, help="Accumulate gradients for ITER_SIZE iterations.")
    parser.add_argument('--context_path', type=str, default="resnet101",
                        help='The context path model you are using, resnet18, resnet101.')
    parser.add_argument('--learning_rate', type=float, default=0.01, help='learning rate used for train')
    parser.add_argument('--data', type=str, default='', help='path of training data')
    parser.add_argument("--data-list", type=str, default='/gdrive/MyDrive/data/Cityscapes/train.txt', help="Path to the file listing the images in the source dataset.")
    parser.add_argument("--data-list-val", type=str, default='/gdrive/MyDrive/data/Cityscapes/val.txt', help="Path to the file listing the image in the test subset.")
    parser.add_argument('--num_workers', type=int, default=4, help='num of workers')
    parser.add_argument('--num_classes', type=int, default=32, help='num of object classes (with void)')
    parser.add_argument("--num-steps", type=int, default=250000, help="Number of training steps.")
    parser.add_argument('--cuda', type=str, default='0', help='GPU ids used for training')
    parser.add_argument('--use_gpu', type=bool, default=True, help='whether to user gpu for training')
    parser.add_argument('--pretrained_model_path', type=str, default=None, help='path to pretrained model')
    parser.add_argument('--save_model_path', type=str, default=None, help='path to save model')
    parser.add_argument('--optimizer', type=str, default='rmsprop', help='optimizer, support rmsprop, sgd, adam')
    parser.add_argument('--loss', type=str, default='crossentropy', help='loss function, dice or crossentropy')
    parser.add_argument("--random-scale", action="store_true", help="Whether to randomly scale the inputs during the training.")
    parser.add_argument("--random-mirror", action="store_true", help="Whether to randomly mirror the inputs during the training.")
    parser.add_argument("--set-type", type=str, default='train', help="choose adaptation set.")

    args = parser.parse_args(params)

    input_size = (int(args.crop_height), int(args.crop_width))
    img_mean = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
    #img_mean=(128, 128, 128)

    # create dataset and dataloader
    train_path = [os.path.join(args.data, 'train'), os.path.join(args.data, 'val')]
    train_label_path = [os.path.join(args.data, 'train_labels'), os.path.join(args.data, 'val_labels')]
    test_path = os.path.join(args.data, 'test')
    test_label_path = os.path.join(args.data, 'test_labels')
    #csv_path = os.path.join(args.data, 'class_dict.csv')
    train_dataset = cityscapesDataSet(train_path, args.data_list,
                    crop_size=input_size,
                    scale=args.random_scale, mirror=args.random_mirror, 
                    mean=img_mean)
    print(f'train_dataset: {len(train_dataset)}')
    
    val_dataset = cityscapesDataSet(test_path, args.data_list_val,
                  crop_size=input_size,
                  scale=False, mirror=args.random_mirror, 
                  mean=img_mean)
    print(f'val_dataset: {len(val_dataset)}')
    
    # Define dataloaders
    dataloader_train = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, pin_memory=True)
    
    dataloader_val = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, pin_memory=True)
    
    # build model
    os.environ['CUDA_VISIBLE_DEVICES'] = args.cuda
    model = BiSeNet(args.num_classes, args.context_path)
    if torch.cuda.is_available() and args.use_gpu:
        model = torch.nn.DataParallel(model).cuda()

    # build optimizer
    if args.optimizer == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), args.learning_rate)
    elif args.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), args.learning_rate, momentum=0.9, weight_decay=1e-4)
    elif args.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), args.learning_rate)
    else:  # rmsprop
        print('not supported optimizer \n')
        return None

    # load pretrained model if exists
    if args.pretrained_model_path is not None:
        print('load model from %s ...' % args.pretrained_model_path)
        model.module.load_state_dict(torch.load(args.pretrained_model_path))
        print('Done!')

    # train
    train(args, model, optimizer, dataloader_train, dataloader_val)

    # val(args, model, dataloader_val, csv_path)

In [11]:
if __name__ == '__main__':
    params = [
        '--num_epochs', '50',
        '--learning_rate', '2.5e-2',
        '--data', '/gdrive/MyDrive/data/Cityscapes',
        '--num_workers', '4',
        '--num_classes', '19',
        '--cuda', '0',
        '--batch_size', '5',
        '--save_model_path', '/gdrive/MyDrive/Project_AML/Models/checkpoints_101_adam',
        '--context_path', 'resnet101',  # set resnet18 or resnet101, only support resnet18 and resnet101
        '--optimizer', 'adam',

    ]
    main(params)

train_dataset: 10
val_dataset: 10


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

...

epoch 10, lr 0.020451:  50%|█████     | 5/10 [00:11<00:11,  2.24s/it, loss=3.958021]

data: tensor([[[[-6.1007e+01, -5.0007e+01, -4.3007e+01,  ..., -5.6007e+01,
           -4.5007e+01, -4.2007e+01],
          [-6.8007e+01, -5.9007e+01, -4.9007e+01,  ..., -5.3007e+01,
           -4.4007e+01, -4.4007e+01],
          [-7.0007e+01, -6.5007e+01, -5.2007e+01,  ..., -5.6007e+01,
           -4.8007e+01, -4.7007e+01],
          ...,
          [-5.3007e+01, -5.2007e+01, -5.2007e+01,  ..., -5.6007e+01,
           -5.7007e+01, -5.7007e+01],
          [-5.3007e+01, -5.2007e+01, -5.2007e+01,  ..., -5.8007e+01,
           -5.7007e+01, -5.8007e+01],
          [-5.3007e+01, -5.2007e+01, -5.2007e+01,  ..., -5.6007e+01,
           -5.7007e+01, -5.9007e+01]],

         [[-5.9669e+01, -4.9669e+01, -4.2669e+01,  ..., -5.7669e+01,
           -4.6669e+01, -4.1669e+01],
          [-6.7669e+01, -5.9669e+01, -4.9669e+01,  ..., -5.4669e+01,
           -4.6669e+01, -4.4669e+01],
          [-6.8669e+01, -6.4669e+01, -5.2669e+01,  ..., -5.5669e+01,
           -5.0669e+01, -4.7669e+01],
          ...,

epoch 10, lr 0.020451: 100%|██████████| 10/10 [00:20<00:00,  2.00s/it, loss=3.646266]


loss for train : 3.802144
start val!


FileNotFoundError: ignored